In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp *.json ~/.cloudvolume/secrets/

In [2]:
!pip install cloud-volume

    100% |################################| 61kB 1.9MB/s ta 0:00:011
    100% |################################| 81kB 3.4MB/s ta 0:00:011
    100% |################################| 133kB 3.1MB/s ta 0:00:01
    100% |################################| 92kB 4.3MB/s ta 0:00:011
    100% |################################| 51kB 6.2MB/s ta 0:00:011
    100% |################################| 92kB 4.5MB/s ta 0:00:011
    100% |################################| 133kB 2.2MB/s ta 0:00:01
    100% |################################| 143kB 4.1MB/s ta 0:00:01
    100% |################################| 419kB 2.6MB/s ta 0:00:01
    100% |################################| 71kB 7.3MB/s eta 0:00:01
    100% |################################| 6.4MB 237kB/s eta 0:00:01
    100% |################################| 194kB 7.0MB/s eta 0:00:01
    100% |################################| 266kB 5.4MB/s eta 0:00:01
    100% |################################| 61kB 12.2MB/s ta 0:00:01
    100% |#####################

    100% |################################| 61kB 12.0MB/s ta 0:00:01
    100% |################################| 61kB 11.9MB/s ta 0:00:01
    100% |################################| 51kB 10.8MB/s ta 0:00:01
    100% |################################| 51kB 11.1MB/s ta 0:00:01
    100% |################################| 61kB 11.2MB/s ta 0:00:01
    100% |################################| 153kB 8.7MB/s eta 0:00:01
    100% |################################| 81kB 13.3MB/s ta 0:00:01
    100% |################################| 51kB 11.7MB/s ta 0:00:01
    100% |################################| 51kB 13.4MB/s ta 0:00:01
    100% |################################| 71kB 12.9MB/s ta 0:00:01
    100% |################################| 92kB 12.2MB/s ta 0:00:01
    100% |################################| 61kB 13.5MB/s ta 0:00:01
    100% |################################| 61kB 12.7MB/s ta 0:00:01
    100% |################################| 51kB 13.4MB/s ta 0:00:01
    100% |#######################

    100% |################################| 51kB 12.0MB/s ta 0:00:01
    100% |################################| 92kB 12.8MB/s ta 0:00:01
    100% |################################| 552kB 2.8MB/s eta 0:00:01
    100% |################################| 194kB 7.3MB/s eta 0:00:01
    100% |################################| 102kB 12.6MB/s a 0:00:01
    100% |################################| 204kB 7.0MB/s eta 0:00:01
    100% |################################| 71kB 11.2MB/s ta 0:00:01
    100% |################################| 51kB 12.2MB/s ta 0:00:01
    100% |################################| 71kB 12.4MB/s ta 0:00:01
    100% |################################| 829kB 1.8MB/s eta 0:00:01
    100% |################################| 8.8MB 176kB/s eta 0:00:01
    100% |################################| 163kB 8.6MB/s eta 0:00:01
    100% |################################| 81kB 12.0MB/s ta 0:00:01


    100% |################################| 225kB 6.2MB/s eta 0:00:01
  Running setup.py bdist_wheel for posix-ipc ... done
  Stored in directory: /root/.cache/pip/wheels/5a/5c/17/d0e2d421abaf3b4097bf9db11108380d734195c6d15c24269d
  Running setup.py bdist_wheel for tenacity ... done
  Stored in directory: /root/.cache/pip/wheels/cf/dd/89/99693d2e2c5954ccd7b7ba4fe9571b385712939d25e584a03a
  Running setup.py bdist_wheel for psutil ... done
  Stored in directory: /root/.cache/pip/wheels/fe/33/84/baea3d665de2d1af7e8f827f3883811bba5e4149443ccf8191
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /root/.cache/pip/wheels/9f/fa/4f/ef45a51543bd855e40494298824ad5e750109ad18fa80278ad
  Running setup.py bdist_wheel for pandocfilters ... done
  Stored in directory: /root/.cache/pip/wheels/fc/a0/4c/ec26eaa3f2253144e29941bd4031c317a69595dd0b665a0779
  Running setup.py bdist_wheel for Markdown ... done
  Stored in directory: /root/.cache/pip/wheels/34/2d/c2/75f4fd57c1b79e5

You are using pip version 9.0.3, however version 10.0.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.nn.functional as F 
import torch.optim as optim

from cloudvolume import CloudVolume
import numpy as np
import pandas as pd

import tifffile

In [2]:
# helper operations 
def conv3x3(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels,
        kernel_size=3, stride=1, padding=1, bias=True)

def maxpool2x2():
    return nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

class UpConv2x2(nn.Module):
    def __init__(self, channels):
        super(UpConv2x2, self).__init__()
        self.upsample = nn.Upsample(scale_factor=2)
        self.conv = nn.Conv2d(channels, channels // 2,
            kernel_size=2, stride=1, padding=0, bias=True)

    def forward(self, x):
        x = self.upsample(x)
        x = F.pad(x, (0,1,0,1))
        x = self.conv(x)
        return x 
        
def concat(xh, xv):
    return torch.cat([xh, xv], dim=1)


# unet blocks
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(ConvBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class DownConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(DownConvBlock, self).__init__()
        self.maxpool = maxpool2x2()
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, x):
        x = self.maxpool(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class UpConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        """
        Args:
            in_channels: number of channels in input (1st) feature map
            out_channels: number of channels in output feature maps
        """
        super(UpConvBlock, self).__init__()
        self.upconv = UpConv2x2(in_channels)
        self.conv1 = conv3x3(in_channels, out_channels)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.conv3 = conv3x3(out_channels, out_channels)

    def forward(self, xh, xv):
        """
        Args:
            xh: torch Variable, activations from same resolution feature maps (gray arrow in diagram)
            xv: torch Variable, activations from lower resolution feature maps (green arrow in diagram)
        """
        xv = self.upconv(xv)
        x = concat(xh, xv)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        return x

class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        fs = [16,32,64,128,256]
        self.conv_in = ConvBlock(1, fs[0])
        self.dconv1 = DownConvBlock(fs[0], fs[1])
        self.dconv2 = DownConvBlock(fs[1], fs[2])
        self.dconv3 = DownConvBlock(fs[2], fs[3])
        self.dconv4 = DownConvBlock(fs[3], fs[4])

        self.uconv1 = UpConvBlock(fs[4], fs[3])
        self.uconv2 = UpConvBlock(fs[3], fs[2])
        self.uconv3 = UpConvBlock(fs[2], fs[1])
        self.uconv4 = UpConvBlock(fs[1], fs[0])
        self.conv_out = conv3x3(fs[0], 1)

        self._initialize_weights()

    def forward(self, x):
        x1 = self.conv_in(x)
        x2 = self.dconv1(x1)
        x3 = self.dconv2(x2)
        x4 = self.dconv3(x3)
        x5 = self.dconv4(x4)
        x6 = self.uconv1(x4, x5)
        x7 = self.uconv2(x3, x6)
        x8 = self.uconv3(x2, x7)
        x9 = self.uconv4(x1, x8)
        x10 = self.conv_out(x9)
        return x10

    def _initialize_weights(self):
        conv_modules = [m for m in self.modules() if isinstance(m, nn.Conv2d)]
        for m in conv_modules:
          n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
          m.weight.data.normal_(0, np.sqrt(2. / n))

# train utils
def np2var(arr, cuda):
    arr = np.expand_dims(np.expand_dims(arr, 0), 0)
    arr = torch.from_numpy(arr)
    arr = Variable(arr)
    if cuda:
        arr = arr.cuda()
    return arr

def var2np(arr):
    arr = arr.data.cpu().numpy()[0,0]
    return arr            

In [3]:
net = torch.load('basil_crackdetector_unet180417')

In [4]:
def detect_cracks(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim):
    chunk_x_start = (src_chunk_dim[0]-dst_chunk_dim[0]) // 2
    chunk_y_start = (src_chunk_dim[1]-dst_chunk_dim[1]) // 2
    chunk_x_stop = chunk_x_start + dst_chunk_dim[0]
    chunk_y_stop = chunk_y_start + dst_chunk_dim[1]
    chunk_slice = slice(chunk_x_start, chunk_x_stop), slice(chunk_y_start, chunk_y_stop)
    dst_size = dst_cv.shape
    x_chunks = int(np.ceil(dst_size[0]/dst_chunk_dim[0]))
    y_chunks = int(np.ceil(dst_size[1]/dst_chunk_dim[1]))
    dst_shape = x_chunks*dst_chunk_dim[0], y_chunks*dst_chunk_dim[1]
    
    src_o = src_cv.voxel_offset
    src_s = src_cv.shape
    src_slice = (slice(src_o[0] - chunk_x_start, src_o[0] - chunk_x_start + x_chunks*src_chunk_dim[0]), 
                slice(src_o[1] - chunk_y_start, src_o[1] - chunk_y_start + y_chunks*src_chunk_dim[1]))
    src_shape = (src_slice[0].stop - src_slice[0].start, src_slice[1].stop - src_slice[1].start)
    
    print('Downloading {}'.format(str(z)))
    src_image = src_cv[src_slice + (z,)].astype(np.float32).reshape(src_shape)
    dst_image = np.zeros(dst_shape, np.uint8)
    
    USECUDA=True

    print('Processing {}'.format(str(z)))
    for x in range(x_chunks):
        for y in range(y_chunks):
            # print(((x,y)))
            x_start = x*dst_chunk_dim[0]
            y_start = y*dst_chunk_dim[1]
            dst_x_stop = x_start + dst_chunk_dim[0]
            dst_y_stop = y_start + dst_chunk_dim[1]
            dst_slice = slice(x_start, dst_x_stop), slice(y_start, dst_y_stop)
            
            src_x_stop = x_start + src_chunk_dim[0]
            src_y_stop = y_start + src_chunk_dim[1]
            src_slice = slice(x_start, src_x_stop), slice(y_start, src_y_stop)
        
            img = src_image[src_slice]
            # print(img.shape)
            img = img - np.mean(img) # normalize inputs
            img = img / (np.std(img) + 1e-6)
            img = np2var(img, cuda=USECUDA)
            o = net(img)
            dst_image[dst_slice] = var2np(o)[chunk_slice].astype(np.uint8)
        
    dst_image = dst_image.astype(np.uint8)
    print('Uploading {}'.format(str(z)))
    dst_cv[:,:,z] = dst_image[:dst_size[0],:dst_size[1],np.newaxis, np.newaxis]

In [7]:
src_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped/permuted', mip=5, bounded=False, fill_missing=True, parallel=1)
dst_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped/permuted/mask/crack_detector_v3', mip=5, cdn_cache=False, bounded=False, autocrop=False, parallel=1)

In [8]:
src_chunk_dim = (512, 512)
dst_chunk_dim = (512-128, 512-128)

In [10]:
for z in range(127,137):
    detect_cracks(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim)

Downloading:   0%|          | 0/72 [00:00<?, ?it/s]

Downloading: 138it [00:04, 18.89it/s]                       


Processing 127


Rechunking image:   4%|▍         | 3/72 [00:00<00:03, 18.78it/s]

Uploading 127


...

Downloading: 134it [00:03, 21.82it/s]                        


Processing 136


Rechunking image:   0%|          | 0/72 [00:00<?, ?it/s]

Uploading 136


Uploading:   0%|          | 0/72 [00:00<?, ?it/s]


In [11]:
src_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped', mip=5, bounded=False, fill_missing=True, parallel=1)
dst_cv = CloudVolume('gs://neuroglancer/basil_v0/raw_image_cropped/mask/crack_detector_v3', mip=5, cdn_cache=False, bounded=False, autocrop=False, parallel=1)

In [12]:
for z in range(137,301):
    detect_cracks(z, src_cv, dst_cv, src_chunk_dim, dst_chunk_dim)

Downloading:  28%|██▊       | 20/72 [00:00<00:00, 199.62it/s]

Downloading: 142it [00:03, 23.74it/s]                        


Processing 137


Rechunking image:   6%|▌         | 4/72 [00:00<00:01, 36.21it/s]

Uploading 137


...

Uploading 272


Downloading:  31%|███       | 22/72 [00:00<00:00, 219.55it/s]

Downloading: 134it [00:03, 20.48it/s]                        


Processing 273


KeyboardInterrupt: 